Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Enrique Davalos"
COLLABORATORS = ""

---

# Homework 2: Generators
## CSE 30 Fall 2020

### Copyright Luca de Alfaro, 2020.  License: CC-BY-NC. 

For how to work on this homework assignment, please refer to the instructions posted on Canvas. 

## Submission

[Please submit to this Google Form](https://forms.gle/kqtB1TS5qNFy1BrT6).

Deadline: **October 13 (Tuesday), 2020, 11pm.**

In [ ]:
#@title Let's define a test helper.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)


## Question 1: Fibonacci numbers

Build a generator that returns the [Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number). 
It should yield, one at a time, the elements of the infinite Fibonacci sequence: 

$$
0, 1, 1, 2, 3, 5, 8, \ldots
$$

In [ ]:
### Exercise: implement a Fibonacci number generator

def fibonacci_generator():
    """Generates all Fibonacci numbers."""
    # YOUR CODE HERE
    a, b = 1, 1
    yield 0
    yield a
    yield b
    while True:
        a, b = b, a+b
        yield b
    # source: https://dev.to/kaelscion/testing-different-fibonacci-generator-techniques-in-python-56k0


In [ ]:
# 10 points. 

### Tests for Fibonacci numbers

r = []
for n in fibonacci_generator():
    r.append(n)
    if n > 100:
        break
check_equal(r, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144])



### Background: Generating Combinations

Given a list containing distinct elements, assume we want to iterate over all pairs of elements of the list.
This is easy to do: 

In [ ]:
def iterate_pairs(elements):
    for i, x in enumerate(elements):
        for y in elements[i + 1:]:
            yield {x, y}


In [ ]:
mylist = [1, 2, 3, 4]
for p in iterate_pairs(mylist):
    print(p)


{1, 2}
{1, 3}
{1, 4}
{2, 3}
{2, 4}
{3, 4}


What if I asked you to iterate over triples of elements?  
You could of course use three nested for loops, instead of just two. 
But at some point, it becomes easier to write the general solution.

Consider a list $L = [x_1, x_2, \ldots, x_n]$ consisting of all distinct elements, and an integer $k$.
Let $C(k, L)$ be the set of subsets of $L$ containing $k$ elements. 
$C(k, L)$ is also known as the _combinations of $L$ in groups of $k$_. 

We will write a function `combinations(k, L)` that computes $C(k, L)$. 
To do so, we will use recursion, and write $C(k, L)$ in terms of $C(k-1, \ldots)$. 
That is, to generate the subsets of length $k$, we will rely on a recursive call (a call to $C$ itself) to generate the subsets of length $k-1$. 

There are three cases, depending on the relation between the size $k$ of the subset we want, and the number $n = |L|$ of elements in $L$. 

1. If $k = n$, we can just return a set containing $L$. 

2. If $k > n$, then obviously $C(k, L) = \emptyset$, where $\emptyset$ denotes the empty set, as we cannot obtain a subset bigger than the set itself.

3. If $k < n$, to generate the $k$-subsets of a list $L$, you pick each element $x$ of $L$ in turn, you consider the list $L'$ of elements following $x$ in $L$, and you return the union of $x$, and of a $k-1$-subset of $L'$.
For those who like the precision of formulas: 

$$
\begin{align*}
C(k, [x_1, \ldots, x_n]) = \{ \, & \{x_1\} \cup C(k - 1, [x_2, \ldots, x_n]),\\
& \{x_2\} \cup C(k-1, [x_3, \ldots, x_n]), \\
& \cdots \\
& \{x_i\} \cup C(k-1, [x_{i+1}, \ldots, x_n]),\\
& \cdots \\
& \{x_n\} \cup C(k-1, \emptyset) \, \} \; .
\end{align*}
$$



## Question 2

Implement a function `combinations(k, elements)` implementing the function $C$ above, that is, returning the set of $k$-elements subsets of `elements`. 

Hint: the difficulty is not in writing the recursion, but in deciding when NOT to recur, and rather, yield a result or the absence of results. 

In [ ]:
### Exercise: Generating Combinations

def combinations(k, elements):
    assert isinstance(elements, list)
    # This can be done in 9 lines of code, and possibly fewer.
    # YOUR CODE HERE
    comb = tuple(elements)
    n = len(comb)
    if k > n:
        return
    indices = list(range(k))
    yield set(comb[i] for i in indices)
    while True:
        for i in reversed(range(k)):
            if indices[i] != i + n - k:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, k):
            indices[j] = indices[j-1] + 1
        yield set(comb[i] for i in indices)
    # source: https://docs.python.org/3/library/itertools.html#itertools.combinations


In [ ]:
# 5 points. 

### Basic tests for Combination Generation

# Let us start from some base cases.
L = [1, 2, 3, 4, 5]

# There are no combinations of 5 elements in groups of 6.
n = 0
for c in combinations(6, L):
    n += 1
check_equal(n, 0)


# There is only one combination of 5 elements in groups of 5: the set itself.
n = 0
for c in combinations(5, L):
    check_equal(c, set(L))
    n += 1
check_equal(n, 1)



In [ ]:
# 5 points. 

### Basic tests 2 for Combination Generation
L = [1, 2, 3, 4, 5]

# There is only one combination of 5 elements in groups of 0: the empty set.
n = 0
for c in combinations(0, L):
    check_equal(c, set())
    n += 1
check_equal(n, 1)



In [ ]:
# 5 points

### Basic tests 3 for Combination Generation
L = [1, 2, 3, 4, 5]

# There is only one combination of 5 elements in groups of 5: the set itself.
n = 0
for c in combinations(5, L):
    check_equal(c, set(L))
    n += 1
check_equal(n, 1)



In [ ]:
# 10 points

### Normal tests for Combination Generator

L = [1, 3, 4, 2, 5]
c2 = combinations(2, L)
c3 = combinations(3, L)
n = 0
ok = False
for c in c2:
    n += 1
    check_equal(len(c), 2)
    if c == {3, 2}:
        ok = True
check_equal(n, 10, msg="n")
check_equal(ok, True, msg="ok")

n = 0
ok = False
for c in c3:
    check_equal(len(c), 3)
    n += 1
    if c == {2, 3, 5}:
        ok = True
check_equal(n, 10)
check_equal(ok, True)

